In [2]:
%reload_ext autoreload
%autoreload 120
%matplotlib inline

In [69]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import scipy.sparse as sp
import pickle
import os

In [4]:
folder = 'instacart_2017_05_01'

In [5]:
products = pd.read_csv(folder + '/' + 'products.csv').set_index('product_id'); products.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [6]:
orders = pd.read_csv(folder + '/' + 'order_products__train.csv'); orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


## Constructing the item-item matrix

In [6]:
order_groups = orders.groupby('order_id')

In [7]:
matrix = {} #sparse
for order_id in order_groups.groups.keys():
    cart = order_groups.get_group(order_id)['product_id'].sort_values().to_list()
    for i, el in enumerate(cart):
        if el not in matrix:
            matrix[el] = {}
        # lower or upper triangle matirx --> save memory (symmetric)
        for j in cart[(i+1):]:
            matrix[el][j] = matrix[el].get(j, 0) + 1

We can remove values that are 1 as they don't give a lot of insight and may take a lot a space

In [8]:
delkeys = []
for i in matrix.keys():
    for k,v in matrix[i].items():
        if v == 1:
            delkeys.append((i,k))

len(delkeys)

4468295

In [10]:
for i,j in delkeys:
    del matrix[i][j]

In [15]:
with open('output/item_matrix.pkl', 'wb') as f:
    pickle.dump(matrix, f)

In [8]:
with open('output/item_matrix.pkl', 'rb') as f:
    matrix = pickle.load(f)   

In [18]:
def get_product(df, prd):
    return df.loc[prd].product_name

In [34]:
mat = sp.dok_matrix((len(products),len(products)), dtype=np.int8)

for ix1, ixx2 in matrix.items():
    for ix2 in ixx2:
        mat[ix1, ix2] = ixx2[ix2]

mat = mat.transpose().tocsr()

In [68]:
def get_all_cocart(sp_mat, pid):
    # sp_mat[pid] column product count, row product count: sp_mat.loc[pid] (index is the product id) *triangular matrix*
    return pd.concat((sp_mat[pid], sp_mat.loc[pid])).dropna()

In [38]:
smat = pd.DataFrame.sparse.from_spmatrix(mat)

In [71]:
def print_cocart(pid):
    print(f'item: {get_product(products, pid)}')
    for idx, val in get_all_cocart(smat, pid).nlargest(10).iteritems():
        if val > 0:
            print(get_product(products, pid), val)

In [60]:
print_cocart(63)

item: Banana & Sweet Potato Organic Teething Wafers
Blueberry & Purple Carrot Teething Wafers 17
Bag of Organic Bananas 15
Organic Strawberries 13
Banana 9
Organic Hass Avocado 8
Organic Baby Spinach 6
Organic D'Anjou Pears 6
Organic Chicken & Apple Sausage 5
Organic Mixed Berry Yogurt & Fruit Snack 5
Limes 5


In [70]:
len(products)

49688

Now we can take this step and reconstruct it with the entire dataset. That will be the objective in notebook 3 and 4 followed by some comments on the result